### Lab_7_Inclass_Assignment
- Name:
- Student ID:


##### Data Integration: Approximate String Matching
1- Implement FROM SCRATCH the approximate string matching functionality using Levenshtein Distance
- Levenshtein Distance between two strings a,b of length |a| and |b| is calculated by creating a matrix of dimensions |a+1|x|b+1| and then filling the matrix by the following equation:<br>
lev<sub>a,b</sub>(i,j) = <br>
a. max(i,j) IF min(i,j)=0<br>
b. minimum of:    
    <ol> <li>lev<sub>a,b</sub>(i-1,j)+1</li>
    <li>lev<sub>a,b</sub>(i,j-1)+1</li>
    <li>lev<sub>a,b</sub>(i-1,j-1)+ 
            *1 if (a<sub>i</sub> != b<sub>j</sub>), 0 otherwise*</li> </ol>

In [2]:
import pandas as pd

In [3]:
def calculate_levenshtein_distance(string_a, string_b):
    size_x = len(string_a) + 1
    size_y = len(string_b) + 1
    matrix = [[0 for _ in range(size_y)] for _ in range(size_x)]
    for x in range(size_x):
        matrix [x][0] = x
    for y in range(size_y):
        matrix [0][y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if string_a[x-1] == string_b[y-1]:
                substitution_cost = 0
            else:
                substitution_cost = 1

            matrix [x][y] = min(
                matrix[x-1][y] + 1,      # deletion
                matrix[x][y-1] + 1,      # insertion
                matrix[x-1][y-1] + substitution_cost  # substitution
            )

    return matrix[size_x - 1][size_y - 1]

In [4]:
calculate_levenshtein_distance("HONDA", "HYUNDAI")

3

In [5]:
calculate_levenshtein_distance("intention", "execution")

5

2- Using the implemented Levenshtein's distance, merge the following dataframes based on the closest match for the "Name" attribute. (You will need to find first the matching pairs)

In [6]:
# First Dataframe
data1 = {'ID1': [1, 2, 3, 4],
         'Name1': ['John Smith', 'Alice Johnson', 'Bob Marley', 'David Doe']}
df1 = pd.DataFrame(data1)
# Second Dataframe
data2 = {'ID2': [101, 102, 103, 104,105],
         'Name2': ['Jon Smith', 'Alyce Jonson', 'Robert Marley', 'Dave D.', 'Rob Marly']}
df2 = pd.DataFrame(data2)
display(df1)
display(df2)

,ID1,Name1
0,1,John Smith
1,2,Alice Johnson
2,3,Bob Marley
3,4,David Doe


,ID2,Name2
0,101,Jon Smith
1,102,Alyce Jonson
2,103,Robert Marley
3,104,Dave D.
4,105,Rob Marly


In [7]:
from itertools import product

# a new dataframe that contains all possible pairs of names
pairs = pd.DataFrame(list(product(df1['Name1'], df2['Name2'])), columns=['Name1', 'Name2'])
pairs


,Name1,Name2
0,John Smith,Jon Smith
1,John Smith,Alyce Jonson
2,John Smith,Robert Marley
3,John Smith,Dave D.
4,John Smith,Rob Marly
5,Alice Johnson,Jon Smith
6,Alice Johnson,Alyce Jonson
7,Alice Johnson,Robert Marley
8,Alice Johnson,Dave D.
9,Alice Johnson,Rob Marly


In [8]:
# Levenshtein distance for each pair
pairs['Distance'] = pairs.apply(lambda row: calculate_levenshtein_distance(row['Name1'], row['Name2']), axis=1)
pairs

,Name1,Name2,Distance
0,John Smith,Jon Smith,1
1,John Smith,Alyce Jonson,11
2,John Smith,Robert Marley,11
3,John Smith,Dave D.,9
4,John Smith,Rob Marly,8
5,Alice Johnson,Jon Smith,12
6,Alice Johnson,Alyce Jonson,2
7,Alice Johnson,Robert Marley,13
8,Alice Johnson,Dave D.,11
9,Alice Johnson,Rob Marly,12


In [9]:
# dataframe sorted by the distance in ascending order
pairs = pairs.sort_values(by='Distance')
pairs

,Name1,Name2,Distance
0,John Smith,Jon Smith,1
14,Bob Marley,Rob Marly,2
6,Alice Johnson,Alyce Jonson,2
12,Bob Marley,Robert Marley,4
18,David Doe,Dave D.,4
4,John Smith,Rob Marly,8
10,Bob Marley,Jon Smith,8
15,David Doe,Jon Smith,9
19,David Doe,Rob Marly,9
3,John Smith,Dave D.,9


In [10]:
# We drop duplicate names, keeping only the pair with the smallest distance
pairs = pairs.drop_duplicates(subset=['Name1'], keep='first')
pairs

,Name1,Name2,Distance
0,John Smith,Jon Smith,1
14,Bob Marley,Rob Marly,2
6,Alice Johnson,Alyce Jonson,2
18,David Doe,Dave D.,4


In [11]:
# We merge the pairs dataframe with the original dataframes
merged_df = pd.merge(df1, pairs, on='Name1')
merged_df = pd.merge(merged_df, df2, left_on='Name2', right_on='Name2')
display(merged_df)

,ID1,Name1,Name2,Distance,ID2
0,1,John Smith,Jon Smith,1,101
1,2,Alice Johnson,Alyce Jonson,2,102
2,3,Bob Marley,Rob Marly,2,105
3,4,David Doe,Dave D.,4,104


In [12]:
merged_df = merged_df.drop(columns=['Distance'])
merged_df

,ID1,Name1,Name2,ID2
0,1,John Smith,Jon Smith,101
1,2,Alice Johnson,Alyce Jonson,102
2,3,Bob Marley,Rob Marly,105
3,4,David Doe,Dave D.,104


3- Using bookings and yachts csv:
<ol>
    <li> Inspect the bookings and yachts csv files.</li>
    <li> Before merging, make sure to keep only bookings with correctly formatted Yacht_IDs, where the Yacht_ID should be in a format of two characters, followed by 3 numbers.</li>
    <li> Combine both dataframes into one dataframe.</li>
    <li> Make sure that all columns are of correct format where it is expected that: <br>
        a. Title, First_Name, Last_Name, Street_Address, Area, Post_Code, Telephone, Yacht_ID, Yacht_Name, Yacht_Make, Yacht_Ref, Depart_Port, Depart_Country<br>
        b. Cust_ID, Built, Length(m), Cabins, Toilets, Guiests, Day_Rate are Numbers <br>
        c. Depart_Date is Date (its original format id dd/mm/yyyy)</li>
    
</ol>

***Note: Data may contain Latin Characters, You may need to use Latin Encoding, drop data where name contains any characters other than normal alphabetics***

In [13]:
# load the bookings and yachts csv files
bookings = pd.read_csv('bookings.csv', encoding='latin1')
yachts = pd.read_csv('yachts.csv', encoding='latin1')

# display the first few rows of the bookings dataframe
display(bookings.head())
print(len(bookings))
display(yachts.head())
print(len(yachts))

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Mr,Finley,Kosar,1580,27 Culmore Cross,Marden,TN21 1ZL,77009001111,12/6/2022,14,SY0182
1,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,SY098
2,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,7/5/2022,10,SY026
3,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,7/5/2022,14,SY118
4,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,23/06/2022,14,SY094


979


,Yacht_Name,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,656
1,Amber Sunrise,Benedetti 37 Mirage,SY002,Pavao Island Marina,Croatia,2014,11.30,3,1,8,431
2,Anne Elizabeth,Marchal Grande 410,SY003,Port Djurak,Croatia,2016,12.35,3,2,8,486
3,Aqua Calma,Benedetti 37 Mirage,SY004,Marina Naupotiri,Greece,2007,11.35,3,1,8,377
4,Arizona Challenger,Marchal Grande 382,SY005,Pavao Island Marina,Croatia,2016,11.23,3,2,8,423


119


In [14]:
import re

# Function to check if Yacht_ID is correctly formatted (two characters followed by 3 numbers)
def is_correct_format(yacht_id):
    pattern = re.compile(r'^[A-Za-z]{2}\d{3}$')
    return bool(pattern.match(yacht_id))


In [15]:
# Yacht_IDs that are correctly formatted
correctly_formatted = bookings['Yacht_ID'].apply(is_correct_format)
correctly_formatted.sum()


976

In [16]:
# rows that are not correctly formatted
display(bookings[~correctly_formatted])

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Mr,Finley,Kosar,1580,27 Culmore Cross,Marden,TN21 1ZL,77009001111,12/6/2022,14,SY0182
31,Mr,Heinrich,Koch,1426,93 Manor Place,Christechurch,DL10 4XU,77009001387,12/8/2022,10,S2Y015
95,Mrs,Tammy,Anderson,1616,33 Wyndham Avenue,Canderbury,SP3 7FR,77009001788,29/08/2022,8,S2Y024


So, we have 3 rows with incorrectly formatted Yacht_ID rows.

In [17]:
# Filter bookings with correctly formatted Yacht_IDs
correct_bookings_df = bookings[bookings['Yacht_ID'].apply(is_correct_format)]
correct_bookings_df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
1,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,SY098
2,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,7/5/2022,10,SY026
3,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,7/5/2022,14,SY118
4,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,23/06/2022,14,SY094
5,Mr,William,Lockhart,1340,9 Dovehouse Street,Aberdovy,GR4 4DG,77009001158,20/08/2022,12,SY032
...,...,...,...,...,...,...,...,...,...,...,...
974,Mr,Lewis,Lawrence,4301,92 High Street,Knaresborough,SY30 0SD,77009009974,7/9/2022,10,SY094
975,Mr,Jakob,Enger,4303,129 Grafton Road,Spalding,PE11 5TN,77009009981,29/05/2022,14,SY048
976,Mr,Benjamin,Goossens,4304,39 Globe Road,Walden,DT23 6HD,77009009987,22/07/2022,14,SY114
977,Mr,Logan,Richardson,4308,127 Whiterock Road,Tattershall,AB41 5FB,77009009988,4/6/2022,28,SY009


In [18]:
# The merge is based on 'Yacht_ID' from bookings and 'Yacht_Ref' from yachts
merged_df = pd.merge(correct_bookings_df, yachts, left_on='Yacht_ID', right_on='Yacht_Ref')
merged_df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
1,Mrs,Lauren,George,1358,85 Havant Arcade,Holywell,BN17 8EH,77009001171,21/08/2022,14,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
2,Mr,Evan,Lewis,2719,106 East Dombey Square,Richmond,CA5 3GN,77009005417,7/5/2022,28,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
3,Mr,Henry,Rhodes,3074,125 Chalfont Avenue,Ely,CT4 7ZT,77009006374,7/6/2022,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
4,Mr,Lucas,Rose,3131,50 Saffron Hill,Petersfield,SA12 4NW,77009006539,5/8/2022,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,Mr,Zachary,Hammond,3045,98 Railway Cottages,Midhurst,PL14 7SP,77009006273,11/8/2022,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
972,Mr,Carl,Hind,3239,15 Cavell Street,Cleeve,SA34 0WZ,77009006944,28/06/2022,28,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
973,Mrs,Magdalena,Schuster,3294,101 Brooklands Park,Shaftesbury,EH6 7BH,77009007222,21/05/2022,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
974,Mr,Shaun,Peretti,3358,146 Atwood Road,Chigwell,LL22 8BT,77009007412,6/6/2022,21,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566


In [19]:
# 'Depart_Date' to datetime format
merged_df['Depart_Date'] = pd.to_datetime(merged_df['Depart_Date'], format='%d/%m/%Y')
merged_df.head()

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,2022-07-26,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
1,Mrs,Lauren,George,1358,85 Havant Arcade,Holywell,BN17 8EH,77009001171,2022-08-21,14,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
2,Mr,Evan,Lewis,2719,106 East Dombey Square,Richmond,CA5 3GN,77009005417,2022-05-07,28,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
3,Mr,Henry,Rhodes,3074,125 Chalfont Avenue,Ely,CT4 7ZT,77009006374,2022-06-07,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
4,Mr,Lucas,Rose,3131,50 Saffron Hill,Petersfield,SA12 4NW,77009006539,2022-08-05,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057


In [20]:
# correct data types for numerical columns
numerical_columns = ['Cust_ID', 'Built', 'Length(m)', 'Cabins', 'Toilets', 'Guests', 'Day_Rate', 'Days']
merged_df[numerical_columns] = merged_df[numerical_columns].apply(pd.to_numeric, errors='coerce')

merged_df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,2022-07-26,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
1,Mrs,Lauren,George,1358,85 Havant Arcade,Holywell,BN17 8EH,77009001171,2022-08-21,14,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
2,Mr,Evan,Lewis,2719,106 East Dombey Square,Richmond,CA5 3GN,77009005417,2022-05-07,28,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
3,Mr,Henry,Rhodes,3074,125 Chalfont Avenue,Ely,CT4 7ZT,77009006374,2022-06-07,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
4,Mr,Lucas,Rose,3131,50 Saffron Hill,Petersfield,SA12 4NW,77009006539,2022-08-05,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,Mr,Zachary,Hammond,3045,98 Railway Cottages,Midhurst,PL14 7SP,77009006273,2022-08-11,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
972,Mr,Carl,Hind,3239,15 Cavell Street,Cleeve,SA34 0WZ,77009006944,2022-06-28,28,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
973,Mrs,Magdalena,Schuster,3294,101 Brooklands Park,Shaftesbury,EH6 7BH,77009007222,2022-05-21,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
974,Mr,Shaun,Peretti,3358,146 Atwood Road,Chigwell,LL22 8BT,77009007412,2022-06-06,21,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566


In [21]:
string_columns = ['Title', 'First_Name', 'Last_Name', 'Street_Address', 'Area', 'Post_Code', 'Telephone', 'Yacht_ID', 'Yacht_Name', 'Yacht_Make', 'Yacht_Ref', 'Depart_Port', 'Depart_Country']
merged_df[string_columns] = merged_df[string_columns].astype(str)

In [22]:
merged_df.dtypes

Title                     object
First_Name                object
Last_Name                 object
Cust_ID                    int64
Street_Address            object
Area                      object
Post_Code                 object
Telephone                 object
Depart_Date       datetime64[ns]
Days                       int64
Yacht_ID                  object
Yacht_Name                object
Yacht_Make                object
Yacht_Ref                 object
Depart_Port               object
Depart_Country            object
Built                      int64
Length(m)                float64
Cabins                     int64
Toilets                    int64
Guests                     int64
Day_Rate                   int64
dtype: object

In [23]:
merged_df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,2022-07-26,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
1,Mrs,Lauren,George,1358,85 Havant Arcade,Holywell,BN17 8EH,77009001171,2022-08-21,14,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
2,Mr,Evan,Lewis,2719,106 East Dombey Square,Richmond,CA5 3GN,77009005417,2022-05-07,28,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
3,Mr,Henry,Rhodes,3074,125 Chalfont Avenue,Ely,CT4 7ZT,77009006374,2022-06-07,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
4,Mr,Lucas,Rose,3131,50 Saffron Hill,Petersfield,SA12 4NW,77009006539,2022-08-05,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,Mr,Zachary,Hammond,3045,98 Railway Cottages,Midhurst,PL14 7SP,77009006273,2022-08-11,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
972,Mr,Carl,Hind,3239,15 Cavell Street,Cleeve,SA34 0WZ,77009006944,2022-06-28,28,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
973,Mrs,Magdalena,Schuster,3294,101 Brooklands Park,Shaftesbury,EH6 7BH,77009007222,2022-05-21,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
974,Mr,Shaun,Peretti,3358,146 Atwood Road,Chigwell,LL22 8BT,77009007412,2022-06-06,21,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
